In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go

In [ ]:
SWING_TRADE_DURATION = 5
SWING_EXPECTED_VOLATILITY = 5
SOURCE_FILE = "./input/PETR4_Daily"

In [ ]:
StockPrices = pd.read_csv("{}.csv".format(SOURCE_FILE))
StockPrices['Counter'] = np.arange(len(StockPrices))

In [ ]:
def get_previous_stock_price(df, x):
    if x['Counter'] > (SWING_TRADE_DURATION - 1):
        return float(df[df['Counter'] == (x['Counter'] - SWING_TRADE_DURATION)]['Price'])
    else:
        return 0

def get_future_stock_price(df, x, max):
    if x['Counter'] < (max - SWING_TRADE_DURATION):
        return float(df[df['Counter'] == (x['Counter'] + SWING_TRADE_DURATION)]['Price'])
    else:
        return 0    
    
def calculate_current_trend(x):
    if x['PreviousStartPrice'] > 0:
        return ((float(x['Price']) / float(x['PreviousStartPrice'])) - 1) * 100
    else:
        return 0;

def get_previous_stock_date(df, x):
    if x['Counter'] > (SWING_TRADE_DURATION - 1):
        return float(df[df['Counter'] == (x['Counter'] - SWING_TRADE_DURATION)]['Date'])
    else:
        return 0

def get_future_stock_date(df, x, max):
    if x['Counter'] < (max - SWING_TRADE_DURATION):
        return float(df[df['Counter'] == (x['Counter'] + SWING_TRADE_DURATION)]['Date'])
    else:
        return 0    
    

def calculate_future_trend(x):
    if x['FutureFinalPrice'] > 0:
        return ((float(x['FutureFinalPrice']) / float(x['Price'])) - 1) * 100
    else:
        return 0;

def calculate_swing_strenght(x):
    if float(x['CurrentTrend']) != 0 and float(x['FutureTrend'] != 0):
        return abs(float(x['CurrentTrend']) - float(x['FutureTrend']))
    else:
        return 0
    
def correct_date(x):
    date_str = str(x['Date'])
    return date_str[0:4] + '-' + date_str[4:6] + '-' + date_str[6:8]    

def calculate_intraday_volatility(df, x):
    current_range = df[df['Counter'] >= (x['Counter'] - (SWING_TRADE_DURATION * 2))]
    current_range = current_range[current_range['Counter'] < x['Counter']]
    return (((current_range['High'] / current_range['Low']) - 1).sum() / (SWING_TRADE_DURATION * 2)) * 100

def change_sign(x,y):
    return  not ((x > 0 and y > 0) or (x < 0 and y < 0))

def is_swing_trade(x):
    return 1 if change_sign(x['FutureTrend'], x['CurrentTrend']) and \
                x['SwingStrength'] > (x['IntradayVolatility'] * SWING_EXPECTED_VOLATILITY) and \
                abs(x['FutureTrend']) >= abs(x['CurrentTrend']) else 0

def clean_swing_trade(df, x):
    if x['IsSwing'] == 1:
        current_range = df[df['Counter'] <= (x['Counter'] + SWING_TRADE_DURATION)]
        current_range = current_range[current_range['Counter'] > x['Counter']]
        if current_range['IsSwing'].sum() > 0:
            return 0
        else:
            return 1
    else:
        return 0


In [ ]:
max_counter = StockPrices['Counter'].max()

StockPrices['CorrectedDate'] = StockPrices.apply( lambda x :  correct_date(x), axis =1 )
StockPrices['PreviousStartPrice'] = StockPrices.apply( lambda x :  get_previous_stock_price(StockPrices, x), axis =1 )
StockPrices['FutureFinalPrice'] = StockPrices.apply( lambda x :  get_future_stock_price(StockPrices, x, max_counter), axis =1 )
StockPrices['PreviousStartDate'] = StockPrices.apply( lambda x :  get_previous_stock_date(StockPrices, x), axis =1 )
StockPrices['FutureFinalDate'] = StockPrices.apply( lambda x :  get_future_stock_date(StockPrices, x, max_counter), axis =1 )

StockPrices['CurrentTrend'] = StockPrices.apply(lambda x : calculate_current_trend(x), axis = 1)


StockPrices['FutureTrend'] = StockPrices.apply(lambda x : calculate_future_trend(x), axis = 1)
StockPrices['SwingStrength'] = StockPrices.apply(lambda x: calculate_swing_strenght(x), axis =1)
StockPrices['IntradayVolatility'] = StockPrices.apply(lambda x: calculate_intraday_volatility(StockPrices, x), axis =1)

StockPrices['IsSwing'] = StockPrices.apply(lambda x: is_swing_trade(x), axis =1)
StockPrices['IsSwing'] = StockPrices.apply(lambda x:clean_swing_trade(StockPrices, x), axis =1)


In [ ]:
output_excel_file='./output/PETR4.Analisys.xlsx'
StockPrices.to_excel(output_excel_file)

In [ ]:
#StockPrices[StockPrices['SwingStrength'] > 13]
StockPrices[StockPrices['IsSwing'] == 1].sort_values(by=['Date'], ascending=[0]).head(20)

In [ ]:
fig = go.Figure(data=[go.Candlestick(x=StockPrices['CorrectedDate'],
                open=StockPrices['Open'],
                high=StockPrices['High'],
                low=StockPrices['Low'],
                close=StockPrices['Price'])])
fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()